In [6]:
'''
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.mlab import PCA
import os

patient_nums = np.load("C:/Users/Ye Xiaoqing/Desktop/TD/patients_nums.npy")
'''

'\nimport cv2\nimport numpy as np\nimport matplotlib.pyplot as plt\nfrom matplotlib.mlab import PCA\nimport os\n\npatient_nums = np.load("C:/Users/Ye Xiaoqing/Desktop/TD/patients_nums.npy")\n'

In [7]:
'''
def get_subdirectory(path):
    
    import os
    
    dirList = []
    files = os.listdir(path)
    for f in files:
        if(os.path.isdir(path + '/' + f)):
            if(f[0] == '.'):
                pass #排除隐藏文件夹
            else:
                dirList.append(path+ '/' + f)
    return dirList

def get_ap(path):
    img_pathList = []
    name = 'arterial phase'
    subpath = get_subdirectory(path)
    pathnumber = len(subpath)
    for i in range(pathnumber):
        img_pathList.append(subpath[i] + '/' + name)
    return img_pathList
    
def get_image(path):
    pathList = []
    for f in os.listdir(path):
        if f.endswith('.png'):
            pathList.append(path+'/'+f)
    return pathList

path = 'C:/Users/Ye Xiaoqing/Desktop/TD/dataset'
path_total = get_ap(path)
'''

In [82]:
'''
patient_index = np.load("index_patient.npy")
index_can = np.load("index_can.npy")
'''

In [1]:
#np.save('mask_all.npy',mask_all)
#np.save('index_patient.npy',index_patient)
#np.save('index_can.npy',index_can)

In [102]:
def feature_ap(img):
    ret,thresh = cv2.threshold(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),127,255,0)
    contours,hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)#得到轮廓信息
    cnt = contours[0]#取第一条轮廓
    print(cnt)

    M = cv2.moments(cnt)#计算第一条轮廓的矩

    #imgnew = cv2.drawContours(img, contours, -1, (0,255,0), 3)#把所有轮廓画出来
    #plt.imshow(imgnew)
    #plt.show()
    #print (M)

    #这两行是计算中心点坐标
    if M['m00'] != 0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
    else:
        cx = int(M['m10']/0.01)
        cy = int(M['m01']/0.01)

    #计算轮廓所包含的面积
    area = cv2.contourArea(cnt)
    # print("area=", area)

    #计算轮廓的周长
    perimeter = cv2.arcLength(cnt,True)
   # print("perimeter", perimeter)

   #轮廓的近似
    #epsilon = 0.02 * perimeter
    #approx = cv2.approxPolyDP(cnt,epsilon,True)
    #imgnew1 = cv2.drawContours(img, approx, -1, (0,0,255), 3)
    
    #cv2.waitKey(0)
    #cv2.waitKey(0)
    #cv2.waitKey(0)
    #cv2.waitKey(0)
   
    return area, perimeter, cnt, cx, cy



In [103]:
# mask_index = index_patient
def feature_ap_whl(path_total, mask_index, index_can):
    
    import math
    from sklearn import decomposition
    import random
    
    clas = len(np.unique(mask_index))
    
    ones_num = np.zeros([clas])
    for k in range(clas):
        ones_num[k] = len(np.where(mask_index == k+1)[0])
            
    index_begin, index_end = 0, 0
    
    csa_max, csa_min = np.zeros([clas]), np.zeros([clas])
    area_super, volumn = np.zeros([clas]), np.zeros([clas])
    cx, cy = np.zeros([clas]), np.zeros([clas])
    contour_xy = np.zeros([clas, 20])

    
    for i in range(clas):
        onm_temp = int(ones_num[i, ])
        index_begin = index_end
        index_end = index_begin + onm_temp
        path_temp = path_total[i]
        path_temp = get_image(path_temp)
        
        index_can_posi = index_can[index_begin : index_end]
        area_temp, perimeter_temp = np.zeros([onm_temp]), np.zeros([onm_temp])       
        cx_temp, cy_temp = np.zeros([onm_temp]), np.zeros([onm_temp])
        cnt_temp_x, cnt_temp_y = np.zeros([]), np.zeros([])

        points = 0
        for j in range(onm_temp):
            c = index_can_posi[j]
            path = path_temp[c]
            img = cv2.imread(path)
            
            area_temp[j], perimeter_temp[j], cnt_local, cx_temp[j], cy_temp[j] = feature_ap(img)   
            
            cnt_temp_x = np.append(cnt_temp_x, cnt_local[:, 0:1, 0:1])
            cnt_temp_y = np.append(cnt_temp_y, cnt_local[:, 0:1, 1:2])
            points = points + cnt_local.shape[0]
            
        csa_max[i] = area_temp.max()
        csa_min[i] = area_temp.min()
        area_super[i] = area_temp.sum()
        volumn[i] = perimeter_temp.sum()
        
        
        rem = onm_temp % 2
        inte = int(onm_temp / 2)
        if rem == 1:
            cx[i] = cx_temp[inte]
            cy[i] = cy_temp[inte]
        else:
            cx[i] = (cx_temp[inte - 1] + cx_temp[inte]) / 2
            cy[i] = (cy_temp[inte - 1] + cy_temp[inte]) / 2
            
        cnt_temp = np.hstack((cnt_temp_x, cnt_temp_y))
        points = int(points)
        rubbish_num = (2*points)%10
        cnt_temp_pca = cnt_temp[0: (2*points)-int(rubbish_num)]
        print(cnt_temp.shape,cnt_temp_pca.shape)
        #rubbish_posi = np.zeros([rubbish_num])
        #seq = range(2 * points)
        
        #if np.unique(rubbish_posi) == rubbish_num:
        #    for r in range(rubbish_num):
        #        index_rubbish = random.randint(0, 2*points - 1)
        #        rubbish_posi[r] = index_rubbish

        #rubbish_posi = rubbish_posi.astype(int)
        #cnt_index = np.delete(seq, rubbish_posi)
        #cnt_temp_pca = cnt_temp[0, cnt_index]
        
        column_num = (2 * points)//10
        print(column_num)
        cnt_temp_ppca = cnt_temp_pca.reshape(10, column_num)
        pca = decomposition.PCA(n_components=2)
        contour_temp_pca = pca.fit_transform(cnt_temp_ppca).ravel()
        contour_xy[i:i+1,:] = contour_temp_pca
        
        
        sh_1, sh_2 = area_temp[0], area_temp[1]
        ch_1, ch_2 = perimeter_temp[0], perimeter_temp[1]
        
        if sh_1 >= sh_2:
            h_h = 1
            area_super_h = sh_1 + ch_1 * h_h
        else:          
            h_h = (sh_1 + 0.5 * ch_1 + 0.5 * ch_2 ) / (sh_2 - sh_1)
            area_super_h = math.sqrt(sh_1 * sh_1 + np.pi * sh_1 * sh_1 * h_h * h_h)
            
        v_h = (1 / 3) * sh_1 * h_h
       
        st_2, st_1 = area_temp[onm_temp-2 : onm_temp]
        ct_2, ct_1 = perimeter_temp[onm_temp-2 : onm_temp]
        if st_2 <= st_1:
            h_t = 1
            area_super_t = st_1 + ct_1 * h_t
        else:
            h_t = (st_1 + 0.5 * ct_1 + 0.5 * ct_2 ) / (st_2 - st_1)
            area_super_t = math.sqrt(st_1 * st_1 + np.pi * st_1 * st_1 * h_t * h_t)
            
        v_t = (1 / 3) * np.pi * h_t

        volumn[i] = volumn[i] + v_h + v_t
        area_super[i] = area_super[i] + area_super_h + area_super_t
    
    feat = np.hstack((contour_xy, csa_max.reshape(clas,1), csa_min.reshape(clas,1), volumn.reshape(clas, 1), cx.reshape(clas, 1), cy.reshape(clas, 1)))
    
    return feat

In [ ]:
def test_2pers():
    index_patient = np.load("index_patient.npy")
    index_can = np.load("index_can.npy")
    
    test_num_1 = len(np.where(index_patient==1)[0])
    test_num_2 = len(np.where(index_patient==2)[0])
    test_num = test_num_1 + test_num_2
    index_patient_test = index_patient[0: test_num]
    index_can_test = index_can[0: test_num]
    path_total_test = path_total[0:2]
    ff = feature_ap_whl(path_total_test, index_patient_test, index_can_test)
    
    return ff

ff = test_2pers()

In [ ]:
ff = feature_ap_whl(path_total, patient_index, index_can)
np.save("C:/Users/Ye Xiaoqing/Desktop/TD/feature_one.npy",ff)

In [32]:
'''
#import numpy as np
#import os
#import cv2

#from PIL import Image
mask_write = mask_temp*255
path_imtest = get_ap("C:/Users/Ye Xiaoqing/Desktop/B_data/dataset_testing/")
index_patient = np.array([0,14,28,30,39,33,28,24,24,29,22,30,29,25,32,22,27,32,15,25,20])
for i in range(20):
    path = path_imtest[i]
    for j in range(index_patient[(i+1)]):
        pathj = path + "/{}.png"
        print(index_patient[0:i+2].sum()-j-1)
        cv2.imwrite(pathj.format(str((10001+j))),mask_write[index_patient[0:i+2].sum()-j-1])
'''

13
12
11
10
9
8
7
6
5
4
3
2
1
0
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
219
218
217
216
215
214
213
212
211
210
209
208
207
206
205
204
203
202
201
200
199
198
197
196
248
247
246
245
244
243
242
241
240
239
238
237
236
235
234
233
232
231
230
229
228
227
226
225
224
223
222
221
220
270
269
268
267
266
265
264
263
262
261
260
259
258
257
256
255
254
253
252
251
250
249
300
299
298
297
296
295
29

In [104]:
path_total = np.load("C:/Users/Ye Xiaoqing/Desktop/B/model_2_data/path_total.npy")
patient_index = np.load("C:/Users/Ye Xiaoqing/Desktop/B/model_2_data/index_patient.npy")
index_can = np.load("C:/Users/Ye Xiaoqing/Desktop/B/model_2_data/index_can.npy")

In [105]:
f1_model1 = feature_ap_whl(path_total, patient_index, index_can)

[[[241 391]]

 [[240 392]]

 [[239 393]]

 [[239 394]]

 [[239 395]]

 [[239 396]]

 [[240 397]]

 [[241 398]]

 [[241 399]]

 [[242 400]]

 [[242 401]]

 [[242 402]]

 [[242 403]]

 [[242 404]]

 [[242 405]]

 [[242 406]]

 [[243 407]]

 [[243 408]]

 [[243 409]]

 [[243 410]]

 [[243 411]]

 [[243 412]]

 [[244 413]]

 [[245 414]]

 [[246 415]]

 [[247 416]]

 [[248 416]]

 [[249 417]]

 [[250 417]]

 [[251 417]]

 [[252 417]]

 [[253 416]]

 [[254 415]]

 [[255 414]]

 [[255 413]]

 [[256 412]]

 [[256 411]]

 [[257 410]]

 [[257 409]]

 [[257 408]]

 [[257 407]]

 [[257 406]]

 [[256 405]]

 [[256 404]]

 [[255 403]]

 [[255 402]]

 [[254 401]]

 [[253 400]]

 [[253 399]]

 [[252 398]]

 [[251 397]]

 [[251 396]]

 [[250 395]]

 [[249 394]]

 [[248 394]]

 [[247 393]]

 [[246 393]]

 [[245 392]]

 [[244 392]]

 [[243 391]]

 [[242 391]]]
[[[238 384]]

 [[237 385]]

 [[236 385]]

 [[236 386]]

 [[235 387]]

 [[235 388]]

 [[235 389]]

 [[235 390]]

 [[234 391]]

 [[234 392]]

 [[234

 [[265 335]]]
[[[265 353]]

 [[264 354]]

 [[263 354]]

 [[262 355]]

 [[261 355]]

 [[261 356]]

 [[260 357]]

 [[260 358]]

 [[259 359]]

 [[259 360]]

 [[259 361]]

 [[260 362]]

 [[260 363]]

 [[261 364]]

 [[261 365]]

 [[262 366]]

 [[263 366]]

 [[264 367]]

 [[265 368]]

 [[266 369]]

 [[267 370]]

 [[268 371]]

 [[268 372]]

 [[269 373]]

 [[270 373]]

 [[271 374]]

 [[272 374]]

 [[273 373]]

 [[274 373]]

 [[275 373]]

 [[276 373]]

 [[277 374]]

 [[278 374]]

 [[279 374]]

 [[280 374]]

 [[281 374]]

 [[282 373]]

 [[283 373]]

 [[284 372]]

 [[285 371]]

 [[286 370]]

 [[286 369]]

 [[287 368]]

 [[287 367]]

 [[288 366]]

 [[288 365]]

 [[288 364]]

 [[289 363]]

 [[289 362]]

 [[289 361]]

 [[288 360]]

 [[288 359]]

 [[287 358]]

 [[286 357]]

 [[285 357]]

 [[284 356]]

 [[283 356]]

 [[282 355]]

 [[281 355]]

 [[280 355]]

 [[279 355]]

 [[278 355]]

 [[277 354]]

 [[276 354]]

 [[275 354]]

 [[274 354]]

 [[273 354]]

 [[272 353]]

 [[271 353]]

 [[270 353]]

 [[269

[[[242 294]]

 [[241 295]]

 [[240 296]]

 [[239 297]]

 [[238 298]]

 [[238 299]]

 [[237 300]]

 [[237 301]]

 [[236 302]]

 [[236 303]]

 [[236 304]]

 [[236 305]]

 [[236 306]]

 [[236 307]]

 [[236 308]]

 [[236 309]]

 [[236 310]]

 [[236 311]]

 [[237 312]]

 [[237 313]]

 [[237 314]]

 [[237 315]]

 [[238 316]]

 [[239 317]]

 [[240 317]]

 [[241 317]]

 [[242 317]]

 [[243 318]]

 [[244 318]]

 [[245 318]]

 [[246 318]]

 [[247 318]]

 [[248 319]]

 [[249 319]]

 [[250 319]]

 [[251 320]]

 [[252 320]]

 [[253 321]]

 [[254 321]]

 [[255 321]]

 [[256 321]]

 [[257 320]]

 [[258 319]]

 [[259 319]]

 [[260 318]]

 [[261 317]]

 [[262 317]]

 [[262 316]]

 [[263 315]]

 [[264 314]]

 [[264 313]]

 [[265 312]]

 [[266 311]]

 [[266 310]]

 [[266 309]]

 [[267 308]]

 [[267 307]]

 [[267 306]]

 [[267 305]]

 [[267 304]]

 [[267 303]]

 [[267 302]]

 [[266 301]]

 [[266 300]]

 [[266 299]]

 [[265 298]]

 [[265 297]]

 [[264 297]]

 [[263 296]]

 [[262 296]]

 [[261 296]]

 [[260

 [[244 347]]]
[[[247 355]]

 [[246 356]]

 [[245 356]]

 [[244 357]]

 [[243 357]]

 [[242 358]]

 [[241 357]]

 [[240 358]]

 [[239 357]]

 [[238 357]]

 [[237 357]]

 [[236 357]]

 [[235 357]]

 [[234 358]]

 [[233 359]]

 [[233 360]]

 [[233 361]]

 [[234 362]]

 [[234 363]]

 [[235 364]]

 [[235 365]]

 [[236 366]]

 [[237 367]]

 [[238 367]]

 [[239 368]]

 [[240 368]]

 [[241 368]]

 [[242 369]]

 [[243 370]]

 [[244 371]]

 [[244 372]]

 [[244 373]]

 [[244 374]]

 [[244 375]]

 [[243 376]]

 [[242 377]]

 [[242 378]]

 [[241 379]]

 [[241 380]]

 [[240 381]]

 [[239 381]]

 [[238 381]]

 [[237 381]]

 [[236 381]]

 [[235 380]]

 [[234 379]]

 [[233 379]]

 [[232 378]]

 [[231 377]]

 [[230 376]]

 [[229 375]]

 [[228 375]]

 [[227 375]]

 [[226 375]]

 [[225 375]]

 [[224 376]]

 [[223 376]]

 [[223 377]]

 [[222 378]]

 [[222 379]]

 [[222 380]]

 [[222 381]]

 [[222 382]]

 [[222 383]]

 [[222 384]]

 [[222 385]]

 [[223 386]]

 [[224 387]]

 [[225 388]]

 [[226 389]]

 [[227

[[[268 300]]

 [[267 301]]

 [[266 301]]

 [[265 301]]

 [[264 302]]

 [[263 302]]

 [[262 302]]

 [[261 303]]

 [[260 303]]

 [[259 303]]

 [[258 304]]

 [[257 304]]

 [[256 304]]

 [[255 305]]

 [[254 305]]

 [[253 306]]

 [[252 307]]

 [[251 308]]

 [[251 309]]

 [[251 310]]

 [[251 311]]

 [[251 312]]

 [[252 313]]

 [[252 314]]

 [[253 315]]

 [[253 316]]

 [[254 317]]

 [[255 318]]

 [[256 319]]

 [[257 320]]

 [[257 321]]

 [[258 322]]

 [[258 323]]

 [[259 324]]

 [[259 325]]

 [[260 326]]

 [[260 327]]

 [[261 328]]

 [[261 329]]

 [[262 330]]

 [[263 331]]

 [[264 331]]

 [[265 331]]

 [[266 331]]

 [[267 331]]

 [[268 331]]

 [[269 330]]

 [[270 330]]

 [[271 329]]

 [[272 328]]

 [[273 327]]

 [[274 326]]

 [[274 325]]

 [[275 324]]

 [[275 323]]

 [[276 322]]

 [[276 321]]

 [[276 320]]

 [[276 319]]

 [[277 318]]

 [[277 317]]

 [[277 316]]

 [[277 315]]

 [[277 314]]

 [[277 313]]

 [[277 312]]

 [[276 311]]

 [[276 310]]

 [[276 309]]

 [[275 308]]

 [[275 307]]

 [[274

[[[251 331]]

 [[250 332]]

 [[249 332]]

 [[248 333]]

 [[247 333]]

 [[246 334]]

 [[245 334]]

 [[244 335]]

 [[243 335]]

 [[242 335]]

 [[241 336]]

 [[240 337]]

 [[239 338]]

 [[239 339]]

 [[238 340]]

 [[237 341]]

 [[236 342]]

 [[235 343]]

 [[234 344]]

 [[234 345]]

 [[234 346]]

 [[233 347]]

 [[233 348]]

 [[233 349]]

 [[233 350]]

 [[234 351]]

 [[234 352]]

 [[234 353]]

 [[235 354]]

 [[236 355]]

 [[236 356]]

 [[237 357]]

 [[238 358]]

 [[239 359]]

 [[240 360]]

 [[241 360]]

 [[242 361]]

 [[243 362]]

 [[244 363]]

 [[245 364]]

 [[245 365]]

 [[246 366]]

 [[247 366]]

 [[248 367]]

 [[249 367]]

 [[250 366]]

 [[250 365]]

 [[250 364]]

 [[250 363]]

 [[250 362]]

 [[249 361]]

 [[249 360]]

 [[249 359]]

 [[250 358]]

 [[250 357]]

 [[250 356]]

 [[250 355]]

 [[251 354]]

 [[251 353]]

 [[252 352]]

 [[253 351]]

 [[254 351]]

 [[255 351]]

 [[256 352]]

 [[257 352]]

 [[258 352]]

 [[259 353]]

 [[260 353]]

 [[261 353]]

 [[262 353]]

 [[263 353]]

 [[263

 [[260 319]]]
[[[239 333]]

 [[239 334]]

 [[238 335]]

 [[238 336]]

 [[239 335]]

 [[240 335]]

 [[241 334]]

 [[242 333]]

 [[241 333]]

 [[240 333]]]
[[[260 328]]

 [[259 329]]

 [[258 330]]

 [[257 330]]

 [[256 331]]

 [[255 332]]

 [[254 333]]

 [[253 334]]

 [[252 335]]

 [[251 335]]

 [[250 336]]

 [[249 337]]

 [[249 338]]

 [[249 339]]

 [[250 340]]

 [[251 341]]

 [[252 341]]

 [[253 342]]

 [[253 343]]

 [[253 344]]

 [[253 345]]

 [[253 346]]

 [[253 347]]

 [[252 348]]

 [[251 349]]

 [[250 350]]

 [[249 350]]

 [[248 351]]

 [[247 352]]

 [[247 353]]

 [[246 354]]

 [[245 354]]

 [[244 354]]

 [[243 354]]

 [[242 355]]

 [[241 354]]

 [[241 353]]

 [[242 352]]

 [[243 351]]

 [[242 350]]

 [[241 350]]

 [[240 350]]

 [[239 350]]

 [[238 350]]

 [[237 350]]

 [[236 350]]

 [[235 351]]

 [[234 351]]

 [[233 352]]

 [[233 353]]

 [[233 354]]

 [[233 355]]

 [[234 356]]

 [[235 356]]

 [[236 355]]

 [[237 355]]

 [[238 354]]

 [[239 354]]

 [[240 354]]

 [[241 354]]

 [[242

 [[253 326]]]
[[[259 306]]

 [[258 307]]

 [[257 307]]

 [[257 308]]

 [[257 309]]

 [[256 310]]

 [[257 311]]

 [[256 312]]

 [[256 313]]

 [[256 314]]

 [[256 315]]

 [[256 316]]

 [[256 317]]

 [[256 318]]

 [[256 319]]

 [[255 320]]

 [[254 321]]

 [[253 322]]

 [[252 322]]

 [[251 322]]

 [[250 321]]

 [[249 321]]

 [[248 321]]

 [[248 322]]

 [[248 323]]

 [[248 324]]

 [[248 325]]

 [[248 326]]

 [[249 327]]

 [[249 328]]

 [[250 329]]

 [[251 330]]

 [[252 331]]

 [[253 331]]

 [[254 331]]

 [[255 331]]

 [[256 332]]

 [[257 331]]

 [[258 331]]

 [[259 331]]

 [[260 331]]

 [[261 330]]

 [[262 330]]

 [[263 329]]

 [[264 329]]

 [[265 328]]

 [[266 327]]

 [[267 327]]

 [[268 326]]

 [[269 325]]

 [[270 325]]

 [[270 324]]

 [[271 323]]

 [[270 322]]

 [[270 321]]

 [[269 320]]

 [[268 320]]

 [[267 319]]

 [[266 319]]

 [[265 318]]

 [[264 317]]

 [[264 316]]

 [[265 315]]

 [[266 314]]

 [[267 313]]

 [[268 312]]

 [[268 311]]

 [[268 310]]

 [[267 309]]

 [[266 308]]

 [[265

 [[275 335]]]
[[[277 349]]

 [[276 350]]

 [[275 350]]

 [[274 351]]

 [[273 352]]

 [[272 352]]

 [[271 353]]

 [[270 354]]

 [[269 355]]

 [[268 355]]

 [[267 355]]

 [[266 355]]

 [[265 355]]

 [[264 355]]

 [[263 354]]

 [[262 354]]

 [[261 354]]

 [[260 354]]

 [[259 354]]

 [[258 355]]

 [[257 355]]

 [[256 356]]

 [[255 357]]

 [[254 358]]

 [[254 359]]

 [[254 360]]

 [[254 361]]

 [[255 362]]

 [[255 363]]

 [[256 364]]

 [[256 365]]

 [[257 366]]

 [[257 367]]

 [[258 368]]

 [[259 369]]

 [[259 370]]

 [[260 370]]

 [[261 371]]

 [[262 371]]

 [[263 372]]

 [[264 372]]

 [[265 372]]

 [[266 373]]

 [[267 373]]

 [[268 373]]

 [[269 373]]

 [[270 373]]

 [[271 373]]

 [[272 373]]

 [[273 372]]

 [[274 372]]

 [[275 371]]

 [[276 370]]

 [[277 369]]

 [[277 368]]

 [[278 367]]

 [[279 366]]

 [[279 365]]

 [[280 364]]

 [[281 363]]

 [[281 362]]

 [[282 361]]

 [[283 360]]

 [[283 359]]

 [[284 358]]

 [[284 357]]

 [[284 356]]

 [[284 355]]

 [[284 354]]

 [[284 353]]

 [[284

[[[253 405]]

 [[252 406]]

 [[251 406]]

 [[250 406]]

 [[249 407]]

 [[249 408]]

 [[249 409]]

 [[250 410]]

 [[251 411]]

 [[252 411]]

 [[253 411]]

 [[254 411]]

 [[255 411]]

 [[256 411]]

 [[257 410]]

 [[258 409]]

 [[258 408]]

 [[257 407]]

 [[256 406]]

 [[255 405]]

 [[254 405]]]
[[[239 402]]

 [[238 403]]

 [[237 403]]

 [[237 404]]

 [[236 405]]

 [[235 406]]

 [[234 407]]

 [[234 408]]

 [[234 409]]

 [[235 410]]

 [[236 411]]

 [[237 411]]

 [[238 412]]

 [[239 412]]

 [[240 413]]

 [[241 413]]

 [[242 413]]

 [[243 414]]

 [[244 414]]

 [[245 414]]

 [[246 414]]

 [[247 414]]

 [[248 414]]

 [[249 414]]

 [[250 414]]

 [[251 413]]

 [[252 413]]

 [[253 413]]

 [[254 413]]

 [[255 413]]

 [[256 413]]

 [[257 412]]

 [[258 411]]

 [[257 410]]

 [[256 410]]

 [[255 410]]

 [[254 409]]

 [[253 409]]

 [[252 408]]

 [[251 407]]

 [[250 406]]

 [[250 405]]

 [[249 404]]

 [[248 404]]

 [[247 403]]

 [[246 403]]

 [[245 402]]

 [[244 402]]

 [[243 402]]

 [[242 402]]

 [[241

In [106]:
np.save("C:/Users/Ye Xiaoqing/Desktop/B/model_2_data/f1_model2", f1_model1)

In [107]:
f1_model1.shape

(20, 25)